In [ ]:
import pandas as pd
import time
from collections import deque, defaultdict
from concurrent.futures import ProcessPoolExecutor
# import dask.dataframe as dd
start_full = time.time()


In [2]:
candidate_path = '/home/it/Desktop/RUN/NEUT_Official/NEUT_R1/OfficialData/R1_App_CandidateProfile.csv'
choice_path = '/home/it/Desktop/RUN/NEUT_Official/NEUT_R1/OfficialData/R1_App_Choice.csv'

eligible_candidates_path = '/home/it/Desktop/RUN/NEUT_Official/NEUT_R1/OfficialData/R1_App_CandidateProfile.csv'

In [3]:
candidate = pd.read_csv(candidate_path)
reordered_columns = ['rollno', 'boardId','stateId', 'domicileId', 'genderId','categoryId', 'subCategoryList','categoryCertificateNo']
candidate = candidate[reordered_columns]
candidate = candidate[(candidate['domicileId']==1) | (candidate['domicileId']=='1') | (candidate['domicileId']=='01')]


choice = pd.read_csv(choice_path)
choice = choice.rename(columns={'rollNo': 'rollno', 'programId':'program_name','instituteId':'program_code','optNo':'ChoiceNumber'})
reordered_columns = ['rollno', 'program_name','program_code','ChoiceNumber']
choice = choice[reordered_columns]

eligible_candidates = pd.read_csv(eligible_candidates_path)
eligible_candidates = eligible_candidates.rename(columns={'rollNo': 'rollno'})


# eligible_candidates = eligible_candidates[eligible_candidates['activityId'] == 26]
eligible_rollnos = eligible_candidates['rollno']

candidate_filtered = pd.merge(eligible_rollnos, candidate, on="rollno", how="inner")
candidate_choice_merge = pd.merge(candidate_filtered, choice, on="rollno", how="inner")

# Save merged DataFrame to CSV
candidate_choice_merge.to_csv('AN_candidate_choice_merge.csv', index=False)
# seat = pd.read_csv(seats_path)
# candidate_choice_merge


In [ ]:
def determine_virtual_categories(row):

    # if row['domicileId'] == 1:  # Andaman and Nicobar
    if row['categoryId'] == 1:  # 1 is for CATEGORY-1
        return [1]
    elif row['categoryId'] == 2:  # 2 is for CATEGORY-2
        return [2]
    elif row['categoryId'] == 4:  # 4 is for CATEGORY-3A
        return [4, 35]
    elif row['categoryId'] == 5:  # 5 is for CATEGORY-3B
        return [5, 35]
    elif row['categoryId'] == 6:  # 6 is for CATEGORY-4
        return [6]
    elif row['categoryId'] == 7:  # 7 is for CATEGORY-5
        return [7]
        # return row['categoryId']
    return []  # Default return value if no conditions are met

    
def determine_virtual_sub_categories(row):
    if "PH:No" in row['subCategoryList'] :
        return ['NO']
    elif "PH:Yes" in row['subCategoryList'] :
        return ['NO', 'PH']
    else:
        return []

    
# def explode_dom(df):
#     df['Dom_V'] = df.apply(determine_virtual_domicile, axis=1)
#     df = df.explode('Dom_V')
#     return df

def explode_cat(df):
    df['Cat_V'] = df.apply(determine_virtual_categories, axis=1)
    df = df.explode('Cat_V')
    return df

def explode_subcat(df):
    df['SubCat_V'] = df.apply(determine_virtual_sub_categories, axis=1)
    df = df.explode('SubCat_V')
    return df

start_time_loading_merging = time.time()

merged_ch_df = pd.read_csv(r'AN_candidate_choice_merge.csv')

# Process the DataFrame
# merged_df1 = explode_dom(merged_ch_df)
merged_df2 = explode_cat(merged_ch_df)
merged_df = explode_subcat(merged_df2)


end_time_loading_merging = time.time()
print(f"Time taken for loading tables and merging and running virtulization: {end_time_loading_merging - start_time_loading_merging:.2f} seconds")

print(f"Now Sorting:")
merged_df.sort_values(by=['rollno', 'ChoiceNumber'], inplace=True)
end_time_sorting = time.time()

print(f"Time taken for Sorting: {end_time_sorting - end_time_loading_merging:.2f} seconds")

print(f"Now creating Virtual count")
merged_df['ChoiceNumber_V'] = merged_df.groupby(['rollno']).cumcount() + 1
end_time_V = time.time()
print(f"Time taken for creating Virtual count: {end_time_V- end_time_sorting:.2f} seconds")

# Select columns to reorder them as needed
reordered_columns = ['rollno',  'program_name', 'program_code', 'boardId',
       'stateId', 'genderId',  
       'categoryCertificateNo','domicileId', 'categoryId', 'Cat_V','subCategoryList', 'SubCat_V','ChoiceNumber','ChoiceNumber_V']

normal_df = merged_df[reordered_columns]
# normal_df.loc[normal_df['Dom_V'] == 'NE', 'Cat_V'] = 11

print(f"Now creating csv file")
normal_df.to_csv('AN_Virtualized_Choices.csv', index=False)
# ds_final_df.to_csv('DS_Virtualized_Choices_R7.csv', index=False)


print(f"Time taken for creating csv file: {time.time() - end_time_V:.2f} seconds")

In [ ]:
merged_df.columns

In [ ]:
end_full = time.time()
total_time= end_full -start_full
print(f"Total time taken  - {total_time}")
print(f" In minutes - {total_time/60}")